# Proyecto Imágenes Laboratorio Inteligencia

### Profesor: Claudio Pérez
### Auxiliar: Juan Pérez
### Integrantes: Matías Osses - Ignacio - Alejandro

El objetivo de este proyecto final es desarrollar e implementar un algoritmo de búsqueda de imágenes similares basado en su contenido (Content Based Image Retrieval, CBIR) utilizando distintos métodos de extracción de características. Para el desarrollo de este proyecto debe descargar la base de datos INRIA Holidays dataset. Debe descargar los archivos jpg1.tar.gz y jpg2.tar.gz que contienen imágenes de consulta para 500 clases distintas y las imágenes que deberán ser almacenadas en el buscador, y que están asociadas a esas 500 clases. La figura 1 muestra las principales etapas de un sistema CBIR.


## Importación de librerías

In [93]:
import os
import copy
import time

import cv2
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import normalize

## Cálculo de vector de características

In [145]:


class Imagen():
    
    def __init__(self, path):
        self.nombre = path.split('\\')[-1]
        self.img = cv2.imread(path)
        self.img = cv2.cvtColor(self.img, cv2.COLOR_BGR2HSV)
        self.histogramas = dict()
        self.asd = dict()
        
    def division(self, n=3):
        nombre = '%sx%s'%(n,n)
        rows, cols, depth = self.img.shape
        histH = []
        histS = []
        histV = []
        for y in range(0, rows, int(rows/n)):
            for x in range(0, cols, int(cols/n)):
                celda = self.img[y:y+n, x:x+n, 0]
                histH.append(np.histogram(celda, bins=8)[0])
                celda = self.img[y:y+n, x:x+n, 1]
                histS.append(np.histogram(celda, bins=12)[0])
                celda = self.img[y:y+n, x:x+n, 2]
                histV.append(np.histogram(celda, bins=3)[0]) 
                
        self.histogramas[nombre+'_H'] = np.array(histH)
        self.histogramas[nombre+'_S'] = np.array(histS)
        self.histogramas[nombre+'_V'] = np.array(histV)
        
    def divisionNoRectangular(self):
        
        reg1 = []
        reg2 = []
        reg3 = []
        reg4 = []
        reg5 = []
        rows, cols, _ =self.img.shape
        
        for x in range(cols):
            for y in range(rows):
                pertenencia = self.elipse(x,y)
                if pertenencia == 5:
                    reg5.append(self.img[y,x,:])
                elif pertenencia == 1:
                    reg1.append(self.img[y,x,:])
                
                elif pertenencia == 2:
                    reg2.append(self.img[y,x,:])
                
                elif pertenencia == 3:
                    reg3.append(self.img[y,x,:])
                
                else:
                    reg4.append(self.img[y,x,:])
        
        reg1 = np.array(reg1)
        reg2 = np.array(reg2)
        reg3 = np.array(reg3)
        reg4 = np.array(reg4)
        reg5 = np.array(reg5)
        canal = ['H', 'S', 'V']
        bins = [8, 12, 3]
        
        for (n,c,b) in zip(range(3),canal,bins):
            self.histogramas['reg1_'+c] = np.histogram(reg1[:,n], bins=b)[0]
            self.histogramas['reg2_'+c] = np.histogram(reg2[:,n], bins=b)[0]
            self.histogramas['reg3_'+c] = np.histogram(reg3[:,n], bins=b)[0]
            self.histogramas['reg4_'+c] = np.histogram(reg4[:,n], bins=b)[0]
            self.histogramas['reg5_'+c] = np.histogram(reg5[:,n], bins=b)[0]
        
    def elipse(self,x,y):
        rows, cols, _ = self.img.shape
        b = rows / 2
        a = cols / 2
        dentro = 1 if (x-a)**2/((a-10)**2) + (y - b)**2/((b-10)**2) < 1 else 0
        
        region1 = int(not dentro and x < a and y < b)
        region2 = 2 * int(not dentro and x >= a and y < b)
        region3 = 3 * int(not dentro and x < a and y >= b)
        region4 = 4 * int(not dentro and x >= a and y >= b)
        region5 = 5 * int(dentro)
        
        return region1 + region2 + region3 + region4 + region5
    
    def calcularVectorCaracteristicas(self):
        self.division(3)
        self.division(4)
        self.division(6)
        self.divisionNoRectangular()
        
        h = np.array([],dtype=int)
        for key in self.histogramas.keys():
            h = np.concatenate([h,self.histogramas[key].reshape(-1)])
        return h
        

In [146]:
path = os.getcwd() + '\jpg'
img = path + '\\130000.jpg'

t = time.time()
a = Imagen(img)
hist = a.calcularVectorCaracteristicas()
t2 = time.time()
print('tiempo de procesamiento para calcut2-t)

6.607273101806641


## Distancias

In [4]:
def euclidiana(x,y):
    return np.sqrt(np.sum(x**2+y**2))

def coseno(x,y): 
    num = np.dot(x,y)
    den = np.sqrt((x**2).sum())*np.sqrt((y**2).sum())
    return num/den

def chi_cuadrado(x,y):
    x_t = x.sum()
    y_t = y.sum()
    c = x + y
    x_esperado = x_t*c/(x_t+y_t)
    y_esperado = y_t*c/(x_t+y_t)
    
    return (((x-x_esperado)**2)/x_esperado).sum() + (((y-y_esperado)**2)/y_esperado).sum()
    
def cramer(x,y):
    return(np.sqrt(chi_cuadrado(x,y)/(x.sum() + y.sum())))

## Content Based Image Retrieval (CBIR)

In [5]:
class CBIR():
    
    def __init__(self, distance):
        self.distance = distance
        self.database = list()
    
    def train(X):
        for image in X:
            self.database.append(Imagen(X).calcularVectorCaracteristicas())
    def evaluar(Input):
        scores = [self.distance(Input, x) for x in self.database]